In [1]:
import requests
import re
from re import sub
import pandas

In [2]:
def ppSqFt_func(lookupTable, inZip, ptype):
    for i in range(len(lookupTable)):
        ppZip = lookupTable.iloc[i][0]
        if str(ppZip) == str(inZip):
            if ptype == 1: #ie townhouse
                thrent = lookupTable.iloc[i][4]
                return [ppZip,inZip,thrent] 
            else:
                ppPrice = lookupTable.iloc[i][3]
                return [ppZip,inZip,ppPrice] 

def irrFunc(price, hoa, taxes, rent):
    #print(price, hoa, taxes, rent)
    #sys.exit()
    
    stem = "http://www.calculator.net/rental-property-calculator.html?cuseloan=yes&cdownpayment=20&cinterest=4.5&cloanterm=30&cothercost=3000&cneedrepair=no&ctaxincrease=3&cinsurance=1200&cinsuranceincrease=3&choaincrease=3&cmaintenance=1000&cmaintenanceincrease=3&cother=200&cotherincrease=3&crentincrease=3&cotherincome=0&cotherincomeincrease=3&cvacancy=5&cmanagement=0&cknowsellprice=no&cappreciation=3&cholding=15&csellcost=8&printit=1&ctype=&x=49&y=25&"
    urlPrice = 'cprice=' + str(price)
    urlHoa = '&choa=' + str(hoa)
    urlTax = '&ctax=' + str(taxes)
    urlRent = '&crent=' + str(rent)
    url = stem + urlPrice + urlHoa + urlTax + urlRent
    r = requests.get(url)
    
    cflow = re.search(r"Cash Flow:(.*?)</b", r.text).group(1)
    cflow = cflow[-9:].replace(">","")
    cflow = cflow.replace("b","")
    cflow = cflow.replace("<","")
    irr = re.search(r"""IRR\):<\/td><td class="bigtext"><font color="green"><b>(.*?)<""", r.text).group(1)
    
    return [cflow, irr, url]

In [3]:
rawDF = pandas.read_csv(r"\\gold.rtgold.nima.mil\home\nde_users_27\clunieit\Profile\Desktop\irr.csv", encoding = "ISO-8859-1")
ppSqFtTBL = pandas.read_csv(r"\\gold.rtgold.nima.mil\home\nde_users_27\clunieit\Profile\Desktop\rentFigures.csv")
rawDF.head(10)

,AA,BB,CC
0,"8910 FORT DR, MANASSAS, VA 20110",NaN,Residential
1,Status: ACTIVE,ML#: MN10112071,"List Price: $345,000.00"
2,Type: Detached,Style: Rambler,Transaction Type: Standard
3,BR: 4 FB: 3 HB: 0,Levels: 2 Fpls: 2,ADC Map Coord: 9F6
4,#Gar/Carpt/Assign: 1/1/,HOA/Condo Fee: /,NaN
5,Lot Acres: .48,Int Sqft: 2756,Yr Blt: 1970
6,Adv Sub: LIBERIA,NaN,"Total Taxes: $5,395.94"
7,"Basement Type: Walkout Stairs, Fully Finished",NaN,NaN
8,"Heat: Baseboard, Hot Water,Natural Gas",NaN,NaN
9,Listing Company: K Realty,NaN,NaN


In [4]:
ppSqFtTBL.head(10)

,RegionName,City,CountyName,condosPPSQFT,SFR rent
0,22191,Woodbridge,Prince William,1.222,1802
1,22193,Woodbridge,Prince William,1.250,1926
2,22304,Alexandria,Alexandria City,1.602,2703
3,20147,Ashburn,Loudoun,1.156,2794
4,22204,Arlington,Arlington,1.990,2915
5,22407,Fredericksburg,Spotsylvania,0.950,1607
6,22201,Arlington,Arlington,2.484,3828
7,22192,Woodbridge,Prince William,1.250,2379
8,22030,Fairfax,Fairfax,1.414,2721
9,22003,Annandale,Fairfax,1.630,2759


In [5]:
import sys
def ppSqFt_func2(lookupTable, inZip, ptype):
    for i in range(len(lookupTable)):
        ppZip = lookupTable.iloc[i][0]
        if str(ppZip) == str(inZip):
            if ptype == 1: #ie townhouse
                thrent = lookupTable.iloc[i][4]
                return [ppZip,inZip,thrent] 
            else:
                ppPrice = lookupTable.iloc[i][3]
                return [ppZip,inZip,ppPrice] 

condos = {}
countr = 0
for i in range(0, len(rawDF), 14):
    print(countr)
    
    adrs = rawDF.iloc[i][0] 
    print(adrs)
    
    zipCode = adrs[-5:]
    print(zipCode)
    

    
    mlsNum = rawDF.iloc[i+1][1]
    #print(mlsNum)
    mlsNum = mlsNum.split(":")[1].replace('\xa0','')
    
    price = rawDF.iloc[i+1][2]
    print(price)
    price = price.split("$")[1].replace('\xa0','')
    price = int(round(float(sub(r'[^\d.]', '', price))))
    
    bedBath = rawDF.iloc[i+3][0]
    print(bedBath)
    bedBath = bedBath.replace('\xa0','')
    
    hoa = rawDF.iloc[i+4][1]
    print(hoa)
    if "$" not in hoa:
        hoa = 400.00
    else:
        hoa = hoa.split("$")[1].replace('\xa0','')
        hoa = hoa.replace('/','')
        hoa = hoa.replace(',','')
        hoa = float(hoa) * 12
    
    sqft = rawDF.iloc[i+5][1]
    print(sqft)
    sqft = sqft.split(":")[1].replace('\xa0','')
    

    if "Townhouse" in rawDF.iloc[i+2][0]:
        rent = ppSqFt_func2(ppSqFtTBL,zipCode,1)[2]
    else:
        ppSqFt = ppSqFt_func2(ppSqFtTBL,zipCode,0)
        print(ppSqFt)
        rent = int(float(sqft) * float(ppSqFt[2]))

        
    yr = rawDF.iloc[i+5][2]
    print(yr)
    yr = yr.split(":")[1].replace('\xa0','')
    
    tax = rawDF.iloc[i+6][2]
    print(tax)
    if "$" not in tax:
        tax = 4000
    else:
        tax = tax.split("$")[1].replace('\xa0','')
        tax = int(round(float(sub(r'[^\d.]', '', tax))))
    
    tempDict = dict({'adrs':adrs,
                       'mls':mlsNum,
                       'zipCode':zipCode,
                       'price':price,
                       'bedBath':bedBath,
                       'hoa':hoa,
                       'sqft':sqft,
                       'ppSqFt':round(ppSqFt[2],2),
                       'yr':yr,
                       'tax':tax,
                       'rent':rent,
                       'cflow':None,
                       'IRR':None,
                       'URL':None})

    countr+=1
    condos[mlsNum] = tempDict
    print()

0
8910 FORT DR, MANASSAS, VA 20110
20110
List Price:  $345,000.00
BR:  4  FB:  3  HB:  0
HOA/Condo Fee:  /
Int Sqft: 2756
[20110, '20110', 1.194]
Yr Blt:  1970
Total Taxes:  $5,395.94

1
15925 DONALD CURTIS DR, WOODBRIDGE, VA 22191
22191
List Price:  $270,000.00
BR:  4  FB:  2  HB:  0
HOA/Condo Fee:  /
Int Sqft: 0
[22191, '22191', 1.222]
Yr Blt:  1967
Total Taxes:  $2,916.14

2
8953 WESTCHESTER DR, MANASSAS, VA 20112
20112
List Price:  $390,000.00
BR:  5  FB:  3  HB:  0
HOA/Condo Fee:  $.00/
Int Sqft: 2221
[20112, '20112', 1.03]
Yr Blt:  1979
Total Taxes:  $3,709.93

3
4817 KEMPAIR CT, WOODBRIDGE, VA 22193
22193
List Price:  $345,000.00
BR:  4  FB:  2  HB:  1
HOA/Condo Fee:  /
Int Sqft: 0
[22193, '22193', 1.25]
Yr Blt:  1973
Total Taxes:  $3,764.90

4
8763 BRADLEY FORGE DR, MANASSAS, VA 20112
20112
List Price:  $380,000.00
BR:  4  FB:  3  HB:  0
HOA/Condo Fee:  /
Int Sqft: 2825
[20112, '20112', 1.03]
Yr Blt:  1982
Total Taxes:  $3,944.15

5
13427 GREENVALE RD, WOODBRIDGE, VA 22192
2219

List Price:  $425,000.00
BR:  4  FB:  2  HB:  1
HOA/Condo Fee:  /
Int Sqft: 0
[22192, '22192', 1.25]
Yr Blt:  1970
Total Taxes:  $4,111.45

46
4821 KENTBURY CT, WOODBRIDGE, VA 22193
22193
List Price:  $364,900.00
BR:  4  FB:  2  HB:  1
HOA/Condo Fee:  /
Int Sqft: 2205
[22193, '22193', 1.25]
Yr Blt:  1973
Total Taxes:  $3,803.14

47
13731 MARILYN CT, WOODBRIDGE, VA 22193
22193
List Price:  $299,000.00
BR:  4  FB:  2  HB:  0
HOA/Condo Fee:  /
Int Sqft: 0
[22193, '22193', 1.25]
Yr Blt:  1979
Total Taxes:  $3,350.24

48
9301 BYRON ST, MANASSAS, VA 20111
20111
List Price:  $319,900.00
BR:  4  FB:  2  HB:  0
HOA/Condo Fee:  /
Int Sqft: 0
[20111, '20111', 1.1859999999999999]
Yr Blt:  1963
Total Taxes:  $3,441.06

49
12404 HARBOR DR, WOODBRIDGE, VA 22192
22192
List Price:  $359,900.00
BR:  4  FB:  2  HB:  1
HOA/Condo Fee:  $.00/
Int Sqft: 0
[22192, '22192', 1.25]
Yr Blt:  1970
Total Taxes:  $3,457.79

50
2171 JENNINGS ST, WOODBRIDGE, VA 22191
22191
List Price:  $359,990.00
BR:  4  FB:  2  HB: 

In [6]:
for key in condos:
    cflwIRRurl = irrFunc(condos[key]['price'], condos[key]['hoa'], condos[key]['tax'], condos[key]['rent'])
    condos[key]['cflow'] = cflwIRRurl[0]
    condos[key]['IRR'] = cflwIRRurl[1]
    condos[key]['URL'] = cflwIRRurl[2]

In [7]:
df = pandas.DataFrame.from_dict(condos)
# df2 = df.transpose
df.to_csv(r"\\gold.rtgold.nima.mil\home\nde_users_27\clunieit\Profile\Desktop\tstOut.csv")